In [1]:
import sys, os, os.path, glob
import tensorflow as tf

In [2]:
project_folder = %pwd
print("Project folder: ", project_folder)
output_folder = os.path.join(project_folder,'output')
data_folder = os.path.join(project_folder, 'data')
train_folder = os.path.join(data_folder, 'train')
validation_folder = os.path.join(data_folder, 'validation')
nb_classes = len(glob.glob(train_folder + "/*"))

Project folder:  D:\fran\Work\eShoConAI_AML


In [5]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="")
    return graph

In [6]:
# We use our "load_graph" function
graph = load_graph(os.path.join(output_folder,'model.pb'))

# We can verify that we can access the list of operations in the graph
for op in graph.get_operations():
    print(op.name)

input_1
conv2d_1/random_uniform/shape
conv2d_1/random_uniform/min
conv2d_1/random_uniform/max
conv2d_1/random_uniform/RandomUniform
conv2d_1/random_uniform/sub
conv2d_1/random_uniform/mul
conv2d_1/random_uniform
conv2d_1/kernel
conv2d_1/kernel/Assign
conv2d_1/kernel/read
conv2d_1/convolution/dilation_rate
conv2d_1/convolution
batch_normalization_1/Const
batch_normalization_1/beta
batch_normalization_1/beta/Assign
batch_normalization_1/beta/read
batch_normalization_1/Const_1
batch_normalization_1/moving_mean
batch_normalization_1/moving_mean/Assign
batch_normalization_1/moving_mean/read
batch_normalization_1/Const_2
batch_normalization_1/moving_variance
batch_normalization_1/moving_variance/Assign
batch_normalization_1/moving_variance/read
batch_normalization_1/moments/mean/reduction_indices
batch_normalization_1/moments/mean
batch_normalization_1/moments/StopGradient
batch_normalization_1/moments/SquaredDifference
batch_normalization_1/moments/variance/reduction_indices
batch_normaliza

batch_normalization_29/batchnorm/add
batch_normalization_29/batchnorm/Rsqrt
batch_normalization_29/batchnorm/mul
batch_normalization_29/batchnorm/mul_1
batch_normalization_29/batchnorm/sub
batch_normalization_29/batchnorm/add_1
batch_normalization_29/AssignMovingAvg/decay
batch_normalization_29/AssignMovingAvg/sub
batch_normalization_29/AssignMovingAvg/mul
batch_normalization_29/AssignMovingAvg
batch_normalization_29/AssignMovingAvg_1/decay
batch_normalization_29/AssignMovingAvg_1/sub
batch_normalization_29/AssignMovingAvg_1/mul
batch_normalization_29/AssignMovingAvg_1
batch_normalization_29/cond/Switch
batch_normalization_29/cond/switch_t
batch_normalization_29/cond/switch_f
batch_normalization_29/cond/pred_id
batch_normalization_29/cond/Switch_1
batch_normalization_29/cond/batchnorm/add/y
batch_normalization_29/cond/batchnorm/add/Switch
batch_normalization_29/cond/batchnorm/add
batch_normalization_29/cond/batchnorm/Rsqrt
batch_normalization_29/cond/batchnorm/mul/Switch
batch_normaliz

activation_71/Relu
conv2d_72/random_uniform/shape
conv2d_72/random_uniform/min
conv2d_72/random_uniform/max
conv2d_72/random_uniform/RandomUniform
conv2d_72/random_uniform/sub
conv2d_72/random_uniform/mul
conv2d_72/random_uniform
conv2d_72/kernel
conv2d_72/kernel/Assign
conv2d_72/kernel/read
conv2d_72/convolution/dilation_rate
conv2d_72/convolution
batch_normalization_72/Const
batch_normalization_72/beta
batch_normalization_72/beta/Assign
batch_normalization_72/beta/read
batch_normalization_72/Const_1
batch_normalization_72/moving_mean
batch_normalization_72/moving_mean/Assign
batch_normalization_72/moving_mean/read
batch_normalization_72/Const_2
batch_normalization_72/moving_variance
batch_normalization_72/moving_variance/Assign
batch_normalization_72/moving_variance/read
batch_normalization_72/moments/mean/reduction_indices
batch_normalization_72/moments/mean
batch_normalization_72/moments/StopGradient
batch_normalization_72/moments/SquaredDifference
batch_normalization_72/moments/va

IsVariableInitialized_341
IsVariableInitialized_342
IsVariableInitialized_343
IsVariableInitialized_344
IsVariableInitialized_345
IsVariableInitialized_346
IsVariableInitialized_347
IsVariableInitialized_348
IsVariableInitialized_349
IsVariableInitialized_350
IsVariableInitialized_351
IsVariableInitialized_352
IsVariableInitialized_353
IsVariableInitialized_354
IsVariableInitialized_355
IsVariableInitialized_356
IsVariableInitialized_357
IsVariableInitialized_358
IsVariableInitialized_359
IsVariableInitialized_360
IsVariableInitialized_361
IsVariableInitialized_362
IsVariableInitialized_363
IsVariableInitialized_364
IsVariableInitialized_365
IsVariableInitialized_366
IsVariableInitialized_367
IsVariableInitialized_368
IsVariableInitialized_369
IsVariableInitialized_370
IsVariableInitialized_371
IsVariableInitialized_372
IsVariableInitialized_373
IsVariableInitialized_374
IsVariableInitialized_375
init
global_average_pooling2d_1/Mean/reduction_indices
global_average_pooling2d_1/Mean
den